In [1]:
import xarray as xr
import numpy as np
import datetime 
import numpy as np
import mpmath as mp
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
from sklearn.preprocessing import StandardScaler
%matplotlib qt

from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
import os 
from os import listdir
from os.path import isfile, join

def get_directory(folder):
    files= [a for a in listdir(folder) if isfile(join(folder,a))]
    
    return files
import os 
from os import listdir
from os.path import isfile, join



## DATA COD

In [2]:
#data_folder="C:/Users/enclab01/Desktop/THESIS/codigos tesis/dataseptiembre"
#data_folder="C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Septiembre2022"
data_folder="C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Agosto2022"
list_of_files=get_directory(data_folder)

#List with the paths
newlist=[]
for item in list_of_files:
    newlist.append(data_folder+'/'+str(item))
print(newlist)

import GOES

MM_final_table=[]
for item in newlist:
    #Importing data
    titles=['Dates_and_Hours', 'COD', 'COD deviations','DQF', 'DQF deviations']
    data= pd.read_table(str(item), header=None, names=titles, sep='\t')
    data2=data.iloc[np.arange(1,len(data)),:] #delete the first row
    data2=data2.reset_index(drop=True)
    MM_final_table.append(data2)

    
'merge the dataframes'
df1=MM_final_table[0]
for i in np.arange(1,len(MM_final_table)):
    df1=pd.concat([df1, MM_final_table[i]])
df1= df1.reset_index() 


df1['COD']=df1['COD'].astype(float)
df1['DQF']=df1['DQF'].astype(float)
df1['COD deviations']=df1['COD deviations'].astype(float)
df1['DQF deviations']=df1['DQF deviations'].astype(float)
'Reemplazar los valores que sean mayores a 50 por 55 por nans'
df1['COD'] = df1['COD'].apply(lambda x: np.nan if x > 50 else x)    
     


'We are only interested after the sunrise'
df1['Dates_and_Hours2']=pd.to_datetime(df1['Dates_and_Hours'])
filtro_inicio = df1['Dates_and_Hours2'].dt.hour >= 12
df1 = df1[filtro_inicio] 
df1= df1.reset_index() 
df1=df1[['Dates_and_Hours','COD', 'COD deviations','DQF', 'DQF deviations']]



'Drop the nans'
check_for_nan_1 = df1['COD'].isnull()
data_copy = df1.copy()
for i in np.arange(0,len(df1)):
    if check_for_nan_1[i]==True:
        data_copy.drop(i, axis=0, inplace=True)
df1=data_copy
df1= df1.reset_index() 


df1=df1[['Dates_and_Hours','COD','COD deviations' ,'DQF', 'DQF deviations']]
# Ordenar en orden ascendente por la columna 'fecha'
df1 = df1.sort_values(by='Dates_and_Hours', ascending=True)  


'Rolling average'
df1['Rolling']=df1['COD'].rolling(10,center=True).mean()    


'1 minute resample'
#Resample
df1['index'] = pd.to_datetime(df1['Dates_and_Hours'])
df1.set_index('index', inplace=True)
df1=df1.resample('1T').mean()
df1['t_utc'] = pd.to_datetime(df1.index.values)
df1= df1.reset_index()

# Ordenar en orden ascendente por la columna 'fecha'
df1 = df1.sort_values(by='t_utc', ascending=True)
  
print('ssss',df1)



# -----------------------------------------------------------------    
'Veamos que pasa con un interpolate'    
    
time2=pd.to_datetime(df1['t_utc'])
# Interpolar los valores NaN en el DataFrame df
df2 = df1['COD'].interpolate(method='linear')
  
COD_values2=df2
COD_values2=COD_values2.astype(float)  

df3 = df1['Rolling'].interpolate(method='linear')
   
COD_values3=df3
COD_values3=COD_values3.astype(float)  

# -----------------------------------------------------------------    
    
'Drop the nans'
check_for_nan_1 = df1['COD'].isnull()
data_copy = df1.copy()
for i in np.arange(0,len(df1)):
    if check_for_nan_1[i]==True:
        data_copy.drop(i, axis=0, inplace=True)
df1=data_copy
df1= df1.reset_index() 

df1=df1[['t_utc','COD','COD deviations' ,'DQF', 'DQF deviations','Rolling']] 



time=pd.to_datetime(df1['t_utc'])
COD_values=df1['COD']
DQF_values=df1['DQF']
COD_deviations=df1['COD deviations']
DQF_deviations=df1['DQF deviations']


['C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Agosto2022/Mes01.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Agosto2022/Mes02.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Agosto2022/Mes03.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Agosto2022/Mes04.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Agosto2022/Mes05.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Agosto2022/Mes06.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Agosto2022/Mes07.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Agosto2022/Mes08.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Agosto2022/Mes09.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Agosto2022/Mes10.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos Point Sur/Agosto2022/Mes11.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos 

In [3]:

print(df1)




                  t_utc       COD  COD deviations  DQF  DQF deviations  \
0   2022-08-01 12:01:00  2.005799        0.301670  1.0        0.000000   
1   2022-08-01 12:06:00  1.487563        0.768959  1.0        0.000000   
2   2022-08-01 12:11:00  1.557760        0.514446  1.0        0.000000   
3   2022-08-01 12:16:00  2.773692        1.226411  1.0        0.000000   
4   2022-08-01 12:21:00  3.322448        0.594318  1.0        0.000000   
..                  ...       ...             ...  ...             ...   
338 2022-08-29 15:41:00  4.224020        0.000000  5.5        0.866025   
339 2022-08-29 16:26:00  4.272852        0.000000  4.5        2.598076   
340 2022-08-29 16:36:00  7.483596        0.000000  4.5        2.598076   
341 2022-08-29 16:51:00  6.182207        0.000000  4.5        2.598076   
342 2022-08-30 12:16:00  2.658935        0.000000  9.5        5.894913   

      Rolling  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
..        ...  

## DATA ALTITUDE

In [4]:
data_folder="C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos San Nicolas/dataseptiembrealtura"
list_of_files=get_directory(data_folder)

#List with the paths
newlist=[]
for item in list_of_files:
    newlist.append(data_folder+'/'+str(item))
print(newlist)

import GOES

MM_final_table=[]
for item in newlist:
    #Importing data
    titles=['Dates_and_Hours', 'HT', 'DQF']
    data= pd.read_table(str(item), header=None, names=titles, sep='\t',parse_dates=['Dates_and_Hours'])
    data2=data.iloc[np.arange(1,len(data)),:] #delete the first row
    data2=data2.reset_index(drop=True)
    MM_final_table.append(data2)


'merge the dataframes'
df2=MM_final_table[0]
for i in np.arange(1,len(MM_final_table)):
    df2=pd.concat([df2, MM_final_table[i]])
df2= df2.reset_index() 


'Drop the nans'
check_for_nan_1 = df2['HT'].isnull()
data_copy = df2.copy()
for i in np.arange(0,len(df2)):
    if check_for_nan_1[i]==True:
        data_copy.drop(i, axis=0, inplace=True)
df2=data_copy
df2= df2.reset_index() 


# Ordenar en orden ascendente por la columna 'fecha'
df2 = df2.sort_values(by='Dates_and_Hours', ascending=True)


df2['HT']=df2['HT'].astype(float)
df2['DQF']=df2['DQF'].astype(float)

# 'Reemplazar los valores que sean mayores a 50 por 55 para que sea mas facil el analisis de datos'
# Reemplazar los valores mayores que 50 por 55 en la columna 'columna_valor'
# df1['COD'] = df1['COD'].apply(lambda x: 55 if x > 50 else x)

df2=df2[['Dates_and_Hours','HT','DQF']]
print(df2)

#time2=pd.to_datetime(df2['Dates_and_Hours'])
HT_values=df2['HT']
DQF_values2=df2['DQF']
 

['C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos San Nicolas/dataseptiembrealtura/Septiembre2022-01.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos San Nicolas/dataseptiembrealtura/Septiembre2022-02.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos San Nicolas/dataseptiembrealtura/Septiembre2022-03.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos San Nicolas/dataseptiembrealtura/Septiembre2022-04.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos San Nicolas/dataseptiembrealtura/Septiembre2022-05.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos San Nicolas/dataseptiembrealtura/Septiembre2022-06.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos San Nicolas/dataseptiembrealtura/Septiembre2022-07.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos San Nicolas/dataseptiembrealtura/Septiembre2022-08.txt', 'C:/Users/enclab01/Desktop/THESIS/codigos tesis/Datos San Nicolas/dataseptiembrealtura/Septiembre2022-09.txt', 

In [5]:
#------------------------------------------------------------------------------
#Resample
df2['index'] = pd.to_datetime(df2['Dates_and_Hours'])
df2.set_index('index', inplace=True)
data2=df2.resample('1T').mean()

data2['t_utc'] = pd.to_datetime(data2.index.values)
data2= data2.reset_index()

altitudekm=[]

for item in data2['HT']:
    altitudekm.append(item/1000)

'----------------------------------------------------------------------------'
fig, ax = plt.subplots(1)
fig.autofmt_xdate()
#plt.plot(time, COD_values,'-x',c='black') 
#plt.plot(time, derivative,'-.',c='red') 
plt.plot(data2['t_utc'],altitudekm,'-x')
 
plt.ylabel('Altitude (Km)')
plt.xlabel('Time (UTC)')
xfmt = mdates.DateFormatter('%b %d %H:%M')
ax.xaxis.set_major_formatter(xfmt)

## ALGORITHM

In [6]:
from scipy.signal import find_peaks
from scipy.signal import find_peaks, savgol_filter


'PEAKS DATOS ORIGINALES'

indicess = find_peaks(COD_values, height=0.00002, prominence=(None, 4000), width=(0.000002, 4000),  rel_height=1)
index=indicess[0]

pir=[]
widths=[]
for m in np.arange(0,len(indicess[1]['peak_heights'])):
    pir.append([indicess[0][m],indicess[1]['prominences'][m],indicess[1]['peak_heights'][m],abs(indicess[1]['right_ips'][m]-indicess[1]['left_ips'][m])])
    widths.append(indicess[1]['widths'][m])    
    
'ROLLING AVERAGE'

#Transform a list into a data column
df = pd.DataFrame({'col':COD_values})
dqfvalues=df['col']
rolling=dqfvalues.rolling(10,center=True).mean()

#'peaks rolling'
#indicesrolling,properties = find_peaks(rolling,prominence=1,width=5,  rel_height=1)
    
    
'peaks rolling tras el resample'
indicesrolling, properties = find_peaks(COD_values3, prominence=1,width=5,  rel_height=1)
print(len(indicesrolling)) 
print(len(properties)) 

10
7


In [7]:
import math
from datetime import datetime, timedelta


left=[]
right=[]
time_left=[]
time_right=[]
minutes=[]
for i in np.arange(0,len(indicesrolling)):
    left.append(math.floor(properties["left_ips"][i]))
    right.append(math.ceil(properties["right_ips"][i]))
    time_left.append(time2[math.floor(properties["left_ips"][i])])
    time_right.append(time2[math.ceil(properties["right_ips"][i])])
    
    diferencia= time2[math.ceil(properties["right_ips"][i])] - time2[math.floor(properties["left_ips"][i])] 
    minutos_diferencia = diferencia.total_seconds()/60
    minutes.append(minutos_diferencia)
    
    
#Create a dataframe with all the variables needed for the analysis 
dataframe = pd.DataFrame({'Time_left':time_left,'Time_right':time_right,'Left':left,'Right':right, 'Minutes': minutes, 'Prominences': properties["prominences"]})  
print(dataframe)

            Time_left          Time_right   Left  Right  Minutes  Prominences
0 2022-08-01 13:38:00 2022-08-01 16:21:00     97    260    163.0    12.337598
1 2022-08-01 16:37:00 2022-08-02 12:46:00    276   1485   1209.0     6.966255
2 2022-08-01 13:08:00 2022-08-05 16:46:00     67   6045   5978.0    13.419463
3 2022-08-02 19:51:00 2022-08-05 16:41:00   1910   6040   4130.0     5.558839
4 2022-08-02 23:56:00 2022-08-05 12:03:00   2155   5762   3607.0     2.248273
5 2022-08-05 12:16:00 2022-08-05 14:19:00   5775   5898    123.0     3.304380
6 2022-08-06 21:58:00 2022-08-28 13:11:00   7797  38950  31153.0     4.446982
7 2022-08-28 13:35:00 2022-08-28 16:11:00  38974  39130    156.0     4.590143
8 2022-08-06 13:14:00 2022-08-29 13:11:00   7273  40390  33117.0    12.404070
9 2022-08-29 12:06:00 2022-08-29 12:40:00  40325  40359     34.0     1.073829


## Histogram

In [8]:
plt.figure()
plt.hist(dataframe["Minutes"],bins=200,range=(0,3150))
plt.xlabel('Length of Cloud Dissipation (minutes)')
plt.ylabel('Count (nº Events)')

Text(0, 0.5, 'Count (nº Events)')

In [9]:
plt.figure()
plt.hist(dataframe["Prominences"],bins=200,range=(0,30))
plt.xlabel('Delta COD')
plt.ylabel('Count (nº Events)')

Text(0, 0.5, 'Count (nº Events)')

## 2D HISTOGRAM

In [10]:
x=dataframe["Minutes"]
xx=x.astype(float)
y=dataframe["Prominences"]
yy=y.astype(float)

xedges=50
yedges=50
xmin=0
xmax=600
ymin=0
ymax=10  

HHH, xedges, yedges= np.histogram2d(xx,yy, bins=(xedges,yedges), range=[[xmin,xmax],[ymin,ymax]])

extent = [xmin,xmax, ymin, ymax]

fig, ax= plt.subplots()

im=ax.imshow(HHH.T ,origin='lower', extent=extent, aspect='auto')
plt.colorbar(im, label='# Dissipation events per bin')
#plt.clim(0,0.001) #This limit shows how to low values looks like

#ax.set_facecolor((0.8,0.8,0.8))
ax.set_facecolor('grey')
plt.xlabel('Length of Cloud Dissipation (minutes)',fontsize=12)
plt.ylabel('COD variations', fontsize=12)

Text(0, 0.5, 'COD variations')

##  OTROS GRAFICOS

In [11]:
from datetime import datetime, timedelta

# Definir dos objetos de fecha y hora
fecha1 = datetime(2023, 9, 11, 10, 30)  # Año, mes, día, hora, minutos
fecha2 = datetime(2023, 9, 11, 12, 45)

# Calcular la diferencia entre las dos fechas y obtener el resultado en minutos
diferencia = fecha2 - fecha1
minutos_diferencia = diferencia.total_seconds() / 60

print(minutos_diferencia)

135.0


In [12]:
'----------------------------------------------------------------------------'

fig, ax = plt.subplots(1)
fig.autofmt_xdate()

#plt.plot(time2, COD_values2,'-x',c='red')
#plt.plot(time2, COD_values3,'-x',c='purple') 

# Crear el gráfico de línea con barras de error
plt.errorbar(time, COD_values, yerr=COD_deviations, linestyle='-', marker='o', capsize=5, label='Datos con error')

plt.plot(time, COD_values,'-x',c='black') 
 
    
#plt.plot(time, derivative,'-.',c='red') 
#plt.plot(time, rolling,'X',c='black') 
 
plt.ylabel('COD Measurements')
plt.xlabel('Time (UTC)')
xfmt = mdates.DateFormatter('%b %d %H:%M')
ax.xaxis.set_major_formatter(xfmt)


In [13]:
import matplotlib.pyplot as plt
import numpy as np

# Tus datos
plt.figure()
x = np.arange(1, 6)  # Valores en el eje x
datos = [1, 2, 3, 4, 5]  # Reemplaza esto con tus propios datos
desviaciones_estandar = [0.1, 0.2, 0.1, 0.3, 0.15]  # Reemplaza esto con tus propias desviaciones estándar

# Crear el gráfico de línea con barras de error
plt.errorbar(x, datos, yerr=desviaciones_estandar, linestyle='-', marker='o', capsize=5, label='Datos con error')

# Personalizar el gráfico
plt.xlabel('Categorías')
plt.ylabel('Datos')
plt.title('Gráfico de Línea con Barras de Error')
plt.xticks(x, ['C1', 'C2', 'C3', 'C4', 'C5'])  # Reemplaza con tus etiquetas de categoría
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.legend()

# Mostrar el gráfico
plt.tight_layout()
plt.show()

In [14]:
COD_values3

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
41771    4.913292
41772    4.913292
41773    4.913292
41774    4.913292
41775    4.913292
Name: Rolling, Length: 41776, dtype: float64

## NUEVO PLOT PARA VISUALIZAR LOS ANCHOS

In [15]:
'----------------------------------------------------------------------------'
sas=np.arange(0,len(time2))
fig, ax = plt.subplots(1)

'Ploteamos el rolling average'
plt.plot( sas , COD_values3,'-x',c='black') 
 
    
plt.ylabel('COD Measurements')
plt.xlabel('Time (UTC)')

plt.vlines(x=sas[indicesrolling], ymin=COD_values3[indicesrolling] - properties["prominences"],
           ymax = COD_values3[indicesrolling], color = "C1")
plt.hlines(y=properties["width_heights"], xmin=properties["left_ips"],
           xmax=properties["right_ips"], color = "C1")

plt.plot(properties["left_ips"], properties["width_heights"],'X',c='red')
plt.plot(properties["right_ips"], properties["width_heights"],'X',c='blue')


# Plotea líneas verticales en cada punto
for i in range(len(left)):
    ax.vlines(left[i], ymin=COD_values3[left[i]], ymax=COD_values3[left[i]]+10, color='purple', linestyle='-', alpha=0.5)
# Plotea líneas verticales en cada punto
for i in range(len(right)):
    ax.vlines(right[i], ymin=COD_values3[right[i]], ymax=COD_values3[right[i]]+10, color='purple', linestyle='-', alpha=0.5)


In [16]:
#FIRST PLOT
plt.figure()
ax2=plt.subplot(212)
figsize=(20,4)
   
plt.plot(time, COD_values,'-x',c='black')     
plt.plot(time, rolling,'-x',c='purple') 
#plt.text(0.01, 0.75, "D)", fontweight="bold", fontsize=13 ,transform=ax7.transAxes)
plt.ylabel('COD (adimensional)',fontsize=13)



#SECOND PLOT

ax1=plt.subplot(211,sharex=ax2)
figsize=(20,4)
plt.plot(time2,HT_values ,'-x',color='black')
plt.plot(time2,10000*np.ones(len(time2)))
plt.ylabel('Altitude (m)',fontsize=13)




ValueError: x and y must have same first dimension, but have shapes (41776,) and (4144,)